# Cooking Recipes dataset

The cooking recipes dataset is a collection of html webpage. On the webpage that host the dataset, we found two perl scripts: one that extract the indredient from the recipes and another that extract the nutrients from a recipe.
In order to use those scripts, we clean the log file that came with the data set to transform it into a list of http link. We then exectute those script with this cleaned file as input and redirect the output into another file that we format in order to use it.

## Cleaning msg.txt

In [1]:
import numpy as np
import pandas as pd
import re

In [ ]:
log = open("data/msg.txt", mode='r', buffering=-1, encoding=None, errors=None, newline=None, closefd=True, opener=None)
clean_log = open("data/msg_clean.txt", mode='w', buffering=-1, encoding=None, errors=None, newline=None, closefd=True, opener=None)

In [ ]:
content = log.readlines()

In [ ]:
for i in range(len(content)):
    line = content[i]
    breaked_line = line.split(' ')
    if len(breaked_line) == 1:
        breaked_line = line.split('\t')
        url = breaked_line[1]
        cleaned_url = url[4:]
        clean_log.write(cleaned_url)
        clean_log.write('\n')
    else:
        url = breaked_line[2]
        cleaned_url = url[4:]
        clean_log.write(cleaned_url)
        clean_log.write('\n')

## Cleaning ingredients.txt

In [2]:
ingredients = open("data/recipeClean/ingredients.txt", mode='r', buffering=-1, encoding="ISO-8859-1", errors=None, newline=None, closefd=True, opener=None)

In [3]:
content = ingredients.readlines()

In [4]:
ids = np.zeros(len(content), dtype=object)
titles = np.zeros(len(content), dtype=object)
ingredients = np.zeros(len(content), dtype=object)
for i in range(len(content)):
    line = content[i].split('\t')
    if len(line) > 4:
        id_recipe, title, ingredient = line[0], line[3], line[4]
        ids[i] = id_recipe
        titles[i] = title
        ingredients[i] = ingredient
    else:
        ids[i] = line[0]
        titles[i] = line[2]
        ingredients[i] = ""

In [5]:
titles_pd = pd.DataFrame(titles)
titles_pd = titles_pd.rename(columns={'0':'Titles'})
titles_pd.head(5)

,0
0,Wild Rice Soup
1,Muffaletta Olive Spread Recipe
2,Holiday Jell-O Recipes By Kraft Foods
3,Thanksgiving Stuffing & Dressing
4,Beef Stew VI Recipe


In [6]:
quantities_reserved = ['teaspoon', 'teaspoons', 'tablespoon', 'tablespoons', 'cup', 'cups', 'pint', 'pints', 'quart', 'quarts', 'gallon', 'gallons', 'pound', 'pounds', 'pouch', 'pouches']

In [7]:
test = ingredients[0]
test1 = ingredients[1]
test2 = ingredients[500]

In [25]:
# Helper function for convert_replace_rational
def convert_rational(ingredient):
    ingredient = ingredient.split(' ')
    quantity = 0
    for i in ingredient:
        for j in range(len(i) - 2):
            if(i[j].isdigit() and i[j + 1] == '/' and i[j + 2].isdigit()):
                quantity = int(i[j]) / float(i[j+2])
    return quantity

def convert_replace_rational(ingredient):
    quantity = convert_rational(ingredient)
    return re.sub(r'[0-9]\/[0-9]', str(quantity), ingredient)

In [26]:
def extract_quantity(ingredient):
    ingredient_to_return = convert_replace_rational(ingredient)
    ingredient = ingredient_to_return
    ingredient = ingredient.replace('(', '').replace(')', '')
    ingredient = ingredient.split(' ')
    quantity = 0
    for i in range(len(ingredient) - 1):
        try:
            x = float(ingredient[i])
        except ValueError:
            pass
        quantity = ingredient[i]
        if([ingredient[i+1] == j for j in quantities_reserved]):
            return quantity, ingredient_to_return.replace(str(quantity), '')
    return quantity, ingredient_to_return.replace(str(quantity), '')

In [57]:
def remove_quantity(ingredient):
    ingredient = ingredient.replace('(', '').replace(')', '').replace('/', ' ')
    for reserved in quantities_reserved:
        if reserved in ingredient:
            ingredient = ingredient.replace(reserved, '')
    ingredient = re.sub(r'(\A| )(s|([0-9]))( |\Z)', '', ingredient)
    return ingredient

In [59]:
breaked_ingredients = test.split('|')
breaked_ingredients

['2 tablespoons butter or margarine',
 '2 medium stalks celery, sliced (1 cup)',
 '1 medium carrot, coarsely shredded (1 cup)',
 '1 medium onion, chopped (1/2 cup)',
 '1 small green bell pepper, chopped (1/2 cup)',
 '3 tablespoons Gold medalÂ® all-purpose flour',
 '1/4 teaspoon pepper',
 '1 pouch (from 10-ounce package) frozen cooked wild rice (1 1/4 cups)',
 '1 cup water',
 '1 can (10 1/2 ounces) condensed chicken broth',
 '1 cup half-and-half',
 '1/3 cup slivered almonds, toasted, if desired',
 '1/4 cup chopped fresh parsley\n']

In [64]:
for ingredient in breaked_ingredients:
    print(remove_quantity(ingredient))

s butter or margarine
medium stalks celery, sliced
medium carrot, coarsely shredded
medium onion, chopped2 
small green bell pepper, chopped2 
s Gold medalÂ® all-purpose flour
4  pepper
 from 10-ounce package frozen cooked wild rice1s
 water
can 102 ounces condensed chicken broth
 half-and-half
3  slivered almonds, toasted, if desired
4  chopped fresh parsley



In [ ]:
breaked_ingredients = test1.split('|')

In [ ]:
breaked_ingredients = test2.split('|')

## Cleaning nutrients.txt